# Fáza 3 - Strojové učenie (v 12. týždni): 20 bodov
Pri dátovej analýze nemusí byť naším cieľom získať len znalosti obsiahnuté v aktuálnych dátach, ale aj natrénovať model, ktorý bude schopný robiť rozumné predikcie pre nové pozorovania pomocou techniky strojového učenia. 
## Jednoduchý klasifikátor na základe závislosti v dátach  (5b)
Naimplementujte OneR algorithm (iné mená: OneRule or 1R), ktorý je jednoduchý klasifikátor tzv. rozhodnutie na základe jedného atribútu. Môžete implementovať aj komplikovanejšie t.j. rozhodnutie na základe kombinácie atribútov.
Algoritmus by mal byť realizovaný na základe závislostí v dátach. Vyhodnoťte klasifikátora pomocou metrík accuracy, precision a recall.
## Trénovanie a vyhodnotenie klasifikátorov strojového učenia (5b)
Na trénovanie využite minimálne jeden stromový algoritmus strojového učenia v scikit-learn.
Vizualizujte natrénované pravidlá.
Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall
Porovnajte ašpoň jeden natrénovaný klasifikátor v scikit-learn s jednoduchým klasifikátorom z prvého kroku.
## Optimalizácia alias hyperparameter tuning (5b)
Preskúmajte hyperparametre Vášho zvoleného klasifikačného algoritmu v druhom kroku a vyskúšajte ich rôzne nastavenie tak, aby ste minimalizovali overfitting (preučenie) a optimalizovali výsledok. 
Vysvetlite, čo jednotlivé hyperparametre robia. Pri nastavovaní hyperparametrov algoritmu využite krížovú validáciu (cross validation) na trénovacej množine.
## Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu (5b) 
Vyhodnotíte Vami zvolené stratégie riešenia projektu z hľadiska classification accuracy: 
Stratégie riešenia chýbajúcich hodnôt a outlierov;
Scaling resp. transformer či zlepší accuracy klasifikácie;
Výber atribútov a výber algoritmov strojového učenia;
Hyperparameter tuning resp. ensemble learning.
Ktoré spôsoby z hore-uvedených bodov sa ukázali ako učinné pre Váš dataset? Hodnotenie podložíte dôkazmi. 

Správa sa odovzdáva v poslednom týždni semestra. Dvojica svojmu cvičiacemu odprezentuje vykonanú fázu v Jupyter Notebooku podľa potreby na cvičení. V notebooku uveďte percentuálny podiel práce členov dvojice. Následne správu elektronicky odovzdá jeden člen z dvojice do systému AIS do štvrtka 15.12.2022 23:59.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import tree
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
  



In [68]:
train = pd.read_csv('training_data.csv',sep="\t")
test = pd.read_csv('testing_data.csv',sep="\t")

# Vytvorenie kopie datasetu
train_copy = train.copy()
test_copy = test.copy()


# Implementing OneRule algorithm based on accuracy, precision and recall

class OneRule:
    def __init__(self, metric='accuracy'):
        self.metric = metric
        self.rule = None
        self.rule_class = None

    def fit(self, X, y):
        classes = np.unique(y)
        best_rule = None
        best_rule_class = None
        best_score = 0
        for c in classes:
            rule = X[y == c].mean()
            score = self.score(X, y, rule, c)
            if score > best_score:
                best_score = score
                best_rule = rule
                best_rule_class = c
        self.rule = best_rule
        self.rule_class = best_rule_class

    def predict(self, X):
        return np.array([self.rule_class] * len(X))

    def score(self, X, y, rule, rule_class):
        y_pred = np.array([rule_class] * len(X))
        print(y_pred)
        y_pred[X < rule] = 1 - rule_class
        if self.metric == 'accuracy':
            return accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return precision_score(y, y_pred)
        elif self.metric == 'recall':
            # use zero_division=0 to avoid warning
            return recall_score(y, y_pred, zero_division="0")
        else:
            raise ValueError('Unknown metric: ' + self.metric)

# Applying OneRule algorithm on dataset "warning"
X = train_copy.drop('warning', axis=1)
y = train_copy['warning']

# For each column in dataset, submit it to OneRule algorithm
for i in range(1,len(X.columns)):
    X_i = X.iloc[:, i]
    X_i = X_i.values
    # Fit OneRule algorithm
    one_rule = OneRule()
    one_rule.fit(X_i, y)
    # Predict values
    y_pred = one_rule.predict(X_i)
    # Print results
    print('Column: ' + X.columns[i])
    print('Rule: ' + str(one_rule.rule))
    print('Rule class: ' + str(one_rule.rule_class))
    print('Accuracy: ' + str(accuracy_score(y, y_pred)))
    print('Precision: ' + str(precision_score(y, y_pred)))
    print('Recall: ' + str(recall_score(y, y_pred)))
    print('')


# Use same algorithm on training dataset, but with tree classifier from skicit-learn
X = train_copy.drop('warning', axis=1)
y = train_copy['warning']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)
y_pred = clf.predict(X)
print('Accuracy: ' + str(accuracy_score(y, y_pred)))
print('Precision: ' + str(precision_score(y, y_pred)))
print('Recall: ' + str(recall_score(y, y_pred)))






[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: PAHs
Rule: 0.5165988938985042
Rule class: 1
Accuracy: 0.5969187675070028
Precision: 0.5969187675070028
Recall: 1.0

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: PM10
Rule: 0.509495374833475
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: O3
Rule: 0.651905470891908
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: NOx
Rule: 0.7173573186124478
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: CH4
Rule: 0.6380420618263979
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: PM2.5
Rule: 0.4806757182580357
Rule class: 1
Accuracy: 0.5969187675070028
Precision: 0.5969187675070028
Recall: 1.0



c:\Users\mattp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mattp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mattp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mattp\AppData\Local

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
